In [4]:
import cv2
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Load dataset and train ML model
data = pd.read_csv("traffic.csv")  # Columns: Time, Date, Day of the week, CarCount, BikeCount, BusCount, TruckCount, Total, Traffic Situation

# Convert categorical 'Day of the week' and 'Traffic Situation' into numerical values
data['DayOfWeek'] = data['Day of the week'].astype('category').cat.codes
data['TrafficSituation'] = data['Traffic Situation'].astype('category').cat.codes

# Define features (X) and target variable (y)
X = data[['CarCount', 'BikeCount', 'BusCount', 'TruckCount', 'Total', 'DayOfWeek', 'TrafficSituation']]
y = data['Total'] * 0.3 + data['TrafficSituation'] * 0.2  # Example: Weighted signal time calculation

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# Save trained model
with open("traffic_model.pkl", "wb") as file:
    pickle.dump(model, file)

# Load pre-trained vehicle detection model
vehicle_cascade = cv2.CascadeClassifier(r"C:\Users\prish\PycharmProjects\PythonProject1\haarcascade_car.xml")

# Function to detect vehicles and estimate density
def detect_vehicles(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    vehicles = vehicle_cascade.detectMultiScale(gray, 1.1, 4)
    vehicle_count = len(vehicles)
    
    for (x, y, w, h) in vehicles:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    return frame, vehicle_count

# Function to adjust traffic signal duration
def adjust_signal_duration(car_count, bike_count, bus_count, truck_count, total, day_of_week, traffic_situation):
    input_data = np.array([[car_count, bike_count, bus_count, truck_count, total, day_of_week, traffic_situation]])
    predicted_time = model.predict(input_data)[0] if hasattr(model, 'predict') else 30
    return max(10, min(90, int(predicted_time)))  # Keep range between 10s-90s

# Start video capture
cap = cv2.VideoCapture("5927708-hd_1080_1920_30fps.mp4")  # Replace with real-time camera stream

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame, car_count = detect_vehicles(frame)
    bike_count = car_count // 2  # Assuming bike count is roughly half of detected cars
    bus_count = car_count // 5  # Assuming bus count is a fraction of cars detected
    truck_count = car_count // 8  # Assuming truck count is lower
    total = car_count + bike_count + bus_count + truck_count
    day_of_week = 2  # Example: Assume Wednesday
    traffic_situation = 1  # Example: Assume moderate traffic
    signal_time = adjust_signal_duration(car_count, bike_count, bus_count, truck_count, total, day_of_week, traffic_situation)
    
    cv2.putText(frame, f"Car Count: {car_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
    cv2.putText(frame, f"Bike Count: {bike_count}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(frame, f"Bus Count: {bus_count}", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    cv2.putText(frame, f"Truck Count: {truck_count}", (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    cv2.putText(frame, f"Signal Time: {signal_time}s", (50, 250), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    cv2.imshow("Traffic Monitoring", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()


C:\Users\prish\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\prish\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\prish\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\prish\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\prish\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\prish\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not ha